In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras import layers, models # pyright: ignore[reportMissingImports]
from tensorflow.keras.applications import EfficientNetB0 # type: ignore
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# =====================
# Step 1: Apply Lung Masks
# =====================
base_infect = "covid_qu_ex/Infection Segmentation Data/Infection Segmentation Data"
base_lung = "covid_qu_ex/Lung Segmentation Data/Lung Segmentation Data"
output_base = "covid_qu_ex_masked"

splits = ["Train", "Val", "Test"]
classes = ["COVID-19", "Normal", "Non-COVID"]

os.makedirs(output_base, exist_ok=True)

for split in splits:
    for cls in classes:
        img_dir = os.path.join(base_infect, split, cls, "images")
        mask_dir = os.path.join(base_lung, split, cls, "lung masks")
        out_dir = os.path.join(output_base, split.lower(), cls)
        os.makedirs(out_dir, exist_ok=True)
        
        for fname in tqdm(os.listdir(img_dir), desc=f"{split}-{cls}"):
            img_path = os.path.join(img_dir, fname)
            mask_path = os.path.join(mask_dir, fname)
            
            if not os.path.exists(mask_path):
                continue
            
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = (mask > 0).astype(np.uint8)
            masked_img = img * mask
            masked_img_rgb = cv2.cvtColor(masked_img, cv2.COLOR_GRAY2BGR)
            cv2.imwrite(os.path.join(out_dir, fname), masked_img_rgb)

Test-Non-COVID: 100%|██████████| 292/292 [00:00<00:00, 480.78it/s]
